<a href="https://colab.research.google.com/github/RaiAnish15/Network/blob/main/MST_MI_logreturn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import KBinsDiscretizer



In [ ]:
import yfinance as yf
def fetch_data(stocks, start_date, end_date):
    """Fetch historical data for a list of stocks."""
    data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']
    return data



In [ ]:
def compute_returns(data):
    """Compute daily log returns from the historical stock data."""
    return np.log(data / data.shift(1)).dropna()

def discretize_data(data, bins=5):
    """Discretize continuous data into discrete bins."""
    discretizer = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='uniform')
    return discretizer.fit_transform(data.reshape(-1, 1)).ravel()

def compute_mutual_information_matrix(returns):
    """Compute the Mutual Information matrix from stock returns."""
    num_stocks = returns.shape[1]

    mi_matrix = np.zeros((num_stocks, num_stocks))

    for i in range(num_stocks):
        for j in range(i + 1, num_stocks):
            X = discretize_data(returns.iloc[:, i].values)
            y = discretize_data(returns.iloc[:, j].values)
            mi = mutual_info_regression(X.reshape(-1, 1), y)
            mi_matrix[i, j] = mi[0]
            mi_matrix[j, i] = mi[0]  # Symmetric matrix

    return pd.DataFrame(mi_matrix, index=returns.columns, columns=returns.columns)

def create_graph_from_mi_matrix(mi_matrix, stock_names):
    """Create a graph from a Mutual Information matrix with stock names as labels."""
    G = nx.Graph()
    num_stocks = mi_matrix.shape[0]

    # Add nodes with labels
    for i in range(num_stocks):
        G.add_node(stock_names[i])

    # Add edges with weights
    for i in range(num_stocks):
        for j in range(i + 1, num_stocks):
            weight = mi_matrix.iloc[i, j]
            G.add_edge(stock_names[i], stock_names[j], weight=weight)

    return G

def plot_graph(G, title="Minimum Spanning Tree"):
    """Plot the graph with the given title."""
    pos = nx.spring_layout(G, seed=42)  # Define positions for visualization
    plt.figure(figsize=(12, 10))
    nx.draw(G, pos, with_labels=True, node_size=3000, node_color='skyblue', font_size=12, font_weight='bold', edge_color='gray', node_shape='o')
    plt.title(title)
    plt.show()

def compute_and_plot_mst(G):
    """Compute and plot MST using Kruskal's and Prim's algorithms."""
    # Compute MST using Kruskal's Algorithm
    mst_kruskal = nx.minimum_spanning_tree(G, algorithm='kruskal')
    mst_graph_kruskal = nx.Graph(mst_kruskal)

    # Compute MST using Prim's Algorithm
    mst_prim = nx.minimum_spanning_tree(G, algorithm='prim')
    mst_graph_prim = nx.Graph(mst_prim)

    # Plot the MST using Kruskal's Algorithm
    plot_graph(mst_graph_kruskal, title="MST Using Kruskal's Algorithm")

    # Plot the MST using Prim's Algorithm
    plot_graph(mst_graph_prim, title="MST Using Prim's Algorithm")



In [ ]:
# Example list of 20 Indian stocks
stocks = ['RELIANCE.BO', 'TCS.BO', 'HDFCBANK.BO', 'INFY.BO', 'ICICIBANK.BO',
          'HINDUNILVR.BO', 'SBIN.BO', 'BHARTIARTL.BO', 'KOTAKBANK.BO', 'LT.BO',
          'WIPRO.BO', 'MARUTI.BO', 'HCLTECH.BO', 'SUNPHARMA.BO', 'AXISBANK.BO',
          'M&M.BO', 'ASIANPAINT.BO', 'ADANIPORTS.BO', 'TATAMOTORS.BO', 'DRREDDY.BO']

# Define the period for data fetching
start_date = '2023-01-01'
end_date = '2024-01-01'

# Fetch historical data
data = fetch_data(stocks, start_date, end_date)



In [ ]:
# Compute the daily log returns from stock data
returns = compute_returns(data)

# Compute the Mutual Information matrix using the log returns
mi_matrix = compute_mutual_information_matrix(returns)

# Invert the MI matrix to prioritize largest MI in MST (use the max MI as the reference)
mi_inverted = mi_matrix.max().max() - mi_matrix

# Create graph from the inverted MI matrix
G = create_graph_from_mi_matrix(mi_inverted, returns.columns)

# Compute and plot MST
compute_and_plot_mst(G)
